# Wilson Cowan Tutorial

**This tutorial was tested with the version `0.0.1-beta2` of NeuroTorch.**

<table class="nt-notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://NeuroTorch.github.io/NeuroTorch/"><img src="https://github.com/NeuroTorch/NeuroTorch/blob/main/images/neurotorch_logo_32px.png?raw=true" width=32px height=32px  />Documentation</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/NeuroTorch/NeuroTorch/blob/main/tutorials/time_series_forecasting_wilson_cowan/tutorial.ipynb"><img src="https://github.com/NeuroTorch/NeuroTorch/blob/main/images/colab_logo_32px.png?raw=true" width=32px height=32px  />Run in Google Colab</a>
</td>
  <td>
    <a target="_blank" href="https://github.com/NeuroTorch/NeuroTorch/blob/main/tutorials/time_series_forecasting_wilson_cowan/tutorial.ipynb"><img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width=32px height=32px />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/NeuroTorch/NeuroTorch/blob/main/tutorials/time_series_forecasting_wilson_cowan/tutorial.ipynb"><img src="https://github.com/NeuroTorch/NeuroTorch/blob/main/images/download_logo_32px.png?raw=true" width=32px height=32px />Download notebook</a>
  </td>
</table>

# Quick setup for Google colab

In [ ]:
#@title Install dependencies {display-mode: "form"}

RunningInCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False

if RunningInCOLAB:
    !git clone https://github.com/NeuroTorch/NeuroTorch.git
    %cd NeuroTorch/
    !git checkout 0.0.1-beta2
else:
    !pip install -r requirements.txt

!pip install git+https://github.com/NeuroTorch/NeuroTorch.git@0.0.1-beta2
!pip install pythonbasictools
!pip install unstable

In this tutorial, we will be learning how to use NeuroTorch to train a recurrent neural network that reproduce time series using the Wilson-Cowan dynamic.

**Plan of this notebook**
1. Brief introduction to the theory behind the Wilson-Cowan dynamic
2. Setup
3. How to create Dataset and dataloader
4. Setting up our data
5. How to run the training
6. Tools to visualise the trained data
7. Stacking multiple layers
8. Callbacks
9. Alternative way to enforce Dale's law

---

### 1 - Wilson-Cowan dynamic

Introduce in 1972 by Wilson HR and Cowan JD, the Wilson-Cowan dynamic performs well when it comes to reproducing and predicting calcium neuronal activity. The equation used is the following :

# <center> $\tau \dot{\vec{x}} = -\vec{x} + (1 - \vec{r} \odot \vec{x}) \odot \sigma({\mathbf{W}\vec{x} - \vec{\mu}})$

We can apply Euler's discretisation. By doing so, we obtain :

# <center> $\vec{x}_{(t+1)} = \vec{x}_{(t)}\left(1- \frac{\Delta t}{\tau}\right) + \frac{\Delta t}{\tau}\left(\left(1 - \vec{r}\odot\vec{x}_{(t)}\right)\odot\sigma\left(\mathbf{W}\vec{x}_{(t)} - \vec{\mu}\right)\right)$


However, in NeuroTorch, we slightly modify this equation to allow to use of batches and to be coherent with format that what use in the rest of the module. In neuroscience, it is common to represent the neuronal activity with a matrix ($N$ x $T$) where $N$ is the number of neurons and $T$ is the number of time step. However, in NeuroTorch, we use the following convention : ($B$ x $T$ x $N$) where $B$ is the number of batch. In this notebook, our goal is to reproduce a specific time series, therefore, our batch size will be $B = 1$. We therefore apply a transpose on our weight matrix and inputs. With this in mind, the Wilson-Cowan becomes :

# <center> $\vec{x}_{(t+1)} = \vec{x}_{(t)}\left(1 - \frac{\Delta t}{\tau}\right) + \frac{\Delta t}{\tau}\left(\left(1 - \vec{r}\odot\vec{x}_{(t)}\right)\odot \sigma\left(\vec{x}_{(t)}\mathbf{W} - \vec{\mu}\right)\right)$

By doing so, we obtain the same result but we are now using an equation that follow what have been done so far in NeuroTorch.

Now, let us define the different parameters of this equation:

- $\vec{x}$ is the neuronal activity at a time $t$ (normalized between 0 and 1) of size (1 x $N$)
- $\Delta t$ is the time step of integration
- $\tau$ is the characteristic time. It represents the time needed for the neuronal activity to decay
- $\vec{r}$ is the refractory time. It represents the time needed for a neuron to fire again once she has already fired of size (1 x $N$)
- $\sigma$ is the sigmoid function
- $W$ is the weight connexion (connectome) of size ($N$ x $N$). $W_{ij}$ tell us how neuron $i$ is connected to neuron $j$
- $\mu$ is the activation threshold of size (1 x $N$). It represents the input needed for a neuron to fire. A neuron with a small $\mu$ will be sensitive to input and will easily fire.

One may notice that hidden states were not discussed here. When working with one layer, this is not an issue. However, at the end of this tutorial, we will show how you can use multiple layers. Hidden states will then need to be discussed there.

---

### 2 - Setup

For those not familiar with Python and virtual environment, you can:

- Follow those steps if you understand french : [venv and pycharm](https://github.com/JeremieGince/TutorielPython-Manuel/tree/master/Environments)

- For english speaker : TODO

In [ ]:
%%capture
!pip install -r time_series_forcasting_wilson_cowan/requirements.txt

If you have a cuda device and want to use it for this tutorial, you can uninstall pytorch with `pip uninstall torch` and re-install it with the right cuda version by generating a command with [PyTorch GetStarted](https://pytorch.org/get-started/locally/) web page. However, for this notebook, we recommend using CPU since we are not using batches.

After setting up the virtual environment, we will need to import the necessary packages.

These are the packages that will be needed for NeuroTorch:

In [ ]:
%%capture
import os
from typing import Optional
import torch
from torch.utils.data import DataLoader, Dataset
import neurotorch as nt
from neurotorch.visualisation.time_series_visualisation import VisualiseKMeans
from neurotorch.regularization.connectome import DaleLawL2
from neurotorch import WilsonCowanLayer
from tutorials.util import GoogleDriveDownloader
from copy import deepcopy
from scipy.ndimage import gaussian_filter1d
import numpy as np

---
### 3 - Dataset & Dataloader

When working with PyTorch, it is important to define a dataset and a dataloader. Of course, you can always use the one that are provided in this module. However, you might be interested in knowing how those simple class are built. With NeuroTorch, multiple datasets and dataloaders are already built-in depending on what you need. Dataset are used to prepare your datas to be used in NeuroTorch. For instance, dataset are used to inform NeuroTorch on the transformation you want to apply to you data or the ratio of datas that will be used to training and to evaluate the quality of the training.

1. First, you begin by creating the constructor
2. You redefine the <code>\_\_len\_\_</code> operator. It returns the size of the dataset
3. You redefine the <code>\_\_getitem__</code> operator. You want to be able to use indexing such as <code> dataset[i] </code> can be use to get the *i*th sample in your dataset. You return the value needed for training and testing.

In our case, we have a dataset of size 1 (one sample) since we only use one time series. Also, when defining <code>\_\_getitem__</code>, we will return the initial condition and the rest of the timeseries since we only have one dataset. We unsqueeze the initiale condition so the tensor goes from size [$N$] to size [1 x $N$]. Remember that, in this dataset, your time series must have the size [$T$ x $N$]

In [ ]:
class WCDataset(Dataset):
	def __init__(self, x):
		self.x = x

	def __len__(self):
		return 1

	def __getitem__(self, item):
		return torch.unsqueeze(self.x[0], dim=0), self.x[1:]

---
### 4 - Setting up our data

This section depend heavily on the way you have your data. For this tutorial, we will use a neuronal activity from a zebra fish. We will take 200 random neurons in our dataset. In this dataset, 406 time steps are present. We therefore need to import our data and select a sample size. From there, we want to applied a gaussian filter and set our array in a size that is compatible with NeuroTorch's convention. This part is left to the user since it heavily depend on the user's data. For this tutorial, we will prepare our data directly in our dataset:

In [ ]:
class WSDataset(Dataset):
	"""
	Generate a dataset of Wilson-Cowan time series.
	This dataset is usefull to reproduce a time series using Wilson-Cowan layers.
	"""
	ROOT_FOLDER = "data/ts/"
	FILE_ID_NAME = {
		"SampleZebrafishData_PaulDeKoninckLab_2020-12-16.npy": "1-3jgAZiNU__NxxhXub7ezAJUqDMFpMCO",
	}

	def __init__(
			self,
			filename: Optional[str] = None,
			sample_size: int = 200,
			smoothing_sigma: float = 10.0,
			device: torch.device = torch.device("cpu"),
			download: bool = True,
			**kwargs
	):
		"""
		:param filename: filename of the dataset to load. If None, download the dataset from google drive.
		:param sample_size: number of neuron to use for training
		:param smoothing_sigma: sigma for the gaussian smoothing
		:param device: device to load the dataset on
		:param download: if True, download the dataset from google drive
		"""
		self.ROOT_FOLDER = kwargs.get("root_folder", self.ROOT_FOLDER)
		if filename is None:
			filename = list(self.FILE_ID_NAME.keys())[0]
			download = True
		path = os.path.join(self.ROOT_FOLDER, filename)
		if download:
			assert filename in self.FILE_ID_NAME, \
				f"File {filename} not found in the list of available files: {list(self.FILE_ID_NAME.keys())}."
			GoogleDriveDownloader(self.FILE_ID_NAME[filename], path, skip_existing=True, verbose=False).download()
		ts = np.load(path)
		n_neurons, n_shape = ts.shape
		sample = np.random.randint(n_neurons, size=sample_size)
		data = ts[sample, :]

		for neuron in range(data.shape[0]):
			data[neuron, :] = gaussian_filter1d(data[neuron, :], sigma=smoothing_sigma)
			data[neuron, :] = data[neuron, :] - np.min(data[neuron, :])
			data[neuron, :] = data[neuron, :] / np.max(data[neuron, :])
		self.original_time_series = data
		self.x = torch.tensor(data.T, dtype=torch.float32, device=device)

	def __len__(self):
		"""
		__len__ is used to get the number of samples in the dataset. Since we are training on the entire time series,
		we only have one sample which is the entire time series hence the length is 1.
		"""
		return 1

	def __getitem__(self, item):
		"""
		return the initial condition and the time series that will be use for training.
		"""
		return torch.unsqueeze(self.x[0], dim=0), self.x[1:]

	@property
	def full_time_series(self):
		return self.x[None, :, :]

	@property
	def original_series(self):
		return self.original_time_series



---
### 5 - Training the data

To reproduce a time series, you can follow those easy steps :

1. Think about the parameters of your training. What kind of optimizer, learning rate, regularisation ...
2. Define your function <code> train_with_params </code>
3. Run the code!

With NeuroTorch, you can easily use the optimizer, loss function, regularisation ... that you want! However, when it comes to reproducing time series, we have selected the one that performs the best. For this tutorial, we will use those.

#### Defining <code> train_with_params </code>

As you can see, until now, very few lines of codes were needed. <code> train_with_params </code> is the only function that must be definie by the user because this is where you tell NeuroTorch the parameters you wish to use. We will walk you through the process, so you can easily recreate this function or even modify to fit your needs. <code> train_with_params </code> takes 3 kind of entries :

- Training related arguments
- Wilson-Cowan dynamic arguments
- device to compute

Generally, you want your argument to be aspect of your training that might change from training to training. For instance, if you're planinng on always using the same learning rate, you might want to remove learning rate from <code> train_with_params </code> and directly add this parameters in the code rather than as an arguments. Of course, the opposite can also be done.

In [ ]:
from typing import Optional

def train_with_params(
		path: str,
		sigma: float = 15.0,
		learning_rate: float = 1e-3,
		n_iterations: int = 100,
		forward_weights: Optional[torch.Tensor or np.ndarray] = None,
		std_weights: float = 1,
		dt: float = 1e-3,
		mu: Optional[float or torch.Tensor or np.ndarray] = 0.0,
		mean_mu: Optional[float] = 0.0,
		std_mu: Optional[float] = 1.0,
		r: Optional[float or torch.Tensor or np.ndarray] = 1.0,
		mean_r: Optional[float] = 1.0,
		std_r: Optional[float] = 1.0,
		tau: float = 1.0,
		learn_mu: bool = False,
		learn_r: bool = False,
		learn_tau: bool = False,
		device: torch.device = torch.device("cpu"),
		hh_init: str = "inputs",
		checkpoint_folder="./checkpoints",
		force_dale_law: bool = True
):
	pass

From there, we can build this function by following these steps :

1. Initialise layer
2. Initialise sequential
3. Initialise trainer

##### Step 1.

When initialising the layer, you can specify the initial parameter of the Wilson-Cowan. If a parameter has not received an argument, it will be generated randomly. The forward weights is the main parameter that is trained. If you want, you can also train the parameter r, $\mu$ and $\tau$. Also, you might want to give an initial forward weight that follow some biological properties to accelerate the training.

In [ ]:
def train_with_params(
		path: str,
		sigma: float = 15.0,
		learning_rate: float = 1e-3,
		n_iterations: int = 100,
		forward_weights: Optional[torch.Tensor or np.ndarray] = None,
		std_weights: float = 1,
		dt: float = 1e-3,
		mu: Optional[float or torch.Tensor or np.ndarray] = 0.0,
		mean_mu: Optional[float] = 0.0,
		std_mu: Optional[float] = 1.0,
		r: Optional[float or torch.Tensor or np.ndarray] = 1.0,
		mean_r: Optional[float] = 1.0,
		std_r: Optional[float] = 1.0,
		tau: float = 1.0,
		learn_mu: bool = False,
		learn_r: bool = False,
		learn_tau: bool = False,
		device: torch.device = torch.device("cpu"),
		hh_init: str = "inputs",
		checkpoint_folder="./checkpoints",
		force_dale_law: bool = True
):
	dataset = WSDataset(path=path, sample_size=200, smoothing_sigma=sigma, device=device)
	x = dataset.full_time_series
	ws_layer = WilsonCowanLayer(
		x.shape[-1], x.shape[-1],
		forward_weights=forward_weights,
		std_weights=std_weights,
		forward_sign=0.5,
		dt=dt,
		r=r,
		mean_r=mean_r,
		std_r=std_r,
		mu=mu,
		mean_mu=mean_mu,
		std_mu=std_mu,
		tau=tau,
		learn_r=learn_r,
		learn_mu=learn_mu,
		learn_tau=learn_tau,
		hh_init=hh_init,
		device=device,
		name="WilsonCowan_layer1",
		force_dale_law=force_dale_law
	).build()
	pass

##### Step 2.

We will now add the sequential model. Here, you want to add the layer you have, the device on which you will compute and the foresight time step. Please note that the foresight time step is the number of time step that will be predicted from the initial condition. Since we are trying to reproduce the time series, we want as much time step as the data. Our foresight time step will therefore be the total time step number of our data minus one (to remove the initial condition). After that, you build the model so that the parameter that will be trained can be initialized. We now have :

In [ ]:
def train_with_params(
		path: str,
		sigma: float = 15.0,
		learning_rate: float = 1e-3,
		n_iterations: int = 100,
		forward_weights: Optional[torch.Tensor or np.ndarray] = None,
		std_weights: float = 1,
		dt: float = 1e-3,
		mu: Optional[float or torch.Tensor or np.ndarray] = 0.0,
		mean_mu: Optional[float] = 0.0,
		std_mu: Optional[float] = 1.0,
		r: Optional[float or torch.Tensor or np.ndarray] = 1.0,
		mean_r: Optional[float] = 1.0,
		std_r: Optional[float] = 1.0,
		tau: float = 1.0,
		learn_mu: bool = False,
		learn_r: bool = False,
		learn_tau: bool = False,
		device: torch.device = torch.device("cpu"),
		hh_init: str = "inputs",
		checkpoint_folder="./checkpoints",
		force_dale_law: bool = True
):
	dataset = WSDataset(path=path, sample_size=200, smoothing_sigma=sigma, device=device)
	x = dataset.full_time_series
	ws_layer = WilsonCowanLayer(
		x.shape[-1], x.shape[-1],
		forward_weights=forward_weights,
		std_weights=std_weights,
		forward_sign=0.5,
		dt=dt,
		r=r,
		mean_r=mean_r,
		std_r=std_r,
		mu=mu,
		mean_mu=mean_mu,
		std_mu=std_mu,
		tau=tau,
		learn_r=learn_r,
		learn_mu=learn_mu,
		learn_tau=learn_tau,
		hh_init=hh_init,
		device=device,
		name="WilsonCowan_layer1",
		force_dale_law=force_dale_law
	).build()

	model = nt.SequentialRNN(
		layers=[ws_layer],
		device=device,
		foresight_time_steps=x.shape[1] - 1,
		checkpoint_folder=checkpoint_folder,
	)
	model.build()
	pass

##### Step 3.

To add the trainer, we simply definie the optimizer for the parameter and the regularisation (if we wish to have regularisation). The following regularisation will be applied on the connectome :

<center> $\min_{W} = Tr(W^T(\alpha * W - (1 - \alpha)W_{EI}))$ </center>

The parameter $\alpha$ is a parameter between 0 and 1 that define whether the regularisation is the dale law ($\alpha = 0$), a reduction of the connectome ($\alpha = 1$) or a combination of both. Also, $W_{EI}$ is a reference matrix that must follow dale's law and will guide the optimizer. Please note that the regularisation of the connectome has a separate optimizer. The loss function that is used here is the proportion of variance :
<center> $pVar = 1 - \frac{MSE(x,y)}{var{(x)}}$ </center>

This loss function is define between $-\infty$ and $1$. We want to maximise it.

To use the Dale's law L2 regularisation, a reference connectome that is already following Dale's law needs to be applied. Also, further down the road, it will be interesting to begin the training with a random matrix that follows Dale's law rather than "just" a random matrix. To do so, you can use the <code> init </code> option. With this, a connectome can be easily initialize. To do, <code> init.dale_ </code> needs a sample connectome with the correct size, the ratio between inhibitory and excitation neurons. Finally, the density of connection can be given.

In [ ]:
nt.init.dale_(torch.zeros(200, 200), inh_ratio=0.5, rho=0.99)

Once this step is complete. We will have a well functioning training.

In [ ]:
def train_with_params(
		sigma: float = 15.0,
		learning_rate: float = 1e-3,
		n_iterations: int = 100,
		forward_weights: Optional[torch.Tensor or np.ndarray] = None,
		std_weights: float = 1,
		dt: float = 1e-3,
		mu: Optional[float or torch.Tensor or np.ndarray] = 0.0,
		mean_mu: Optional[float] = 0.0,
		std_mu: Optional[float] = 1.0,
		r: Optional[float or torch.Tensor or np.ndarray] = 1.0,
		mean_r: Optional[float] = 1.0,
		std_r: Optional[float] = 1.0,
		tau: float = 1.0,
		learn_mu: bool = False,
		learn_r: bool = False,
		learn_tau: bool = False,
		device: torch.device = torch.device("cpu"),
		hh_init: str = "inputs",
		checkpoint_folder="./checkpoints",
		force_dale_law: bool = True
):
	# STEP 1
	dataset = WSDataset(sample_size=200, smoothing_sigma=sigma, device=device)
	x = dataset.full_time_series
	ws_layer = WilsonCowanLayer(
		x.shape[-1], x.shape[-1],
		forward_weights=forward_weights,
		std_weights=std_weights,
		forward_sign=0.5,
		dt=dt,
		r=r,
		mean_r=mean_r,
		std_r=std_r,
		mu=mu,
		mean_mu=mean_mu,
		std_mu=std_mu,
		tau=tau,
		learn_r=learn_r,
		learn_mu=learn_mu,
		learn_tau=learn_tau,
		hh_init=hh_init,
		device=device,
		name="WilsonCowan_layer1",
		force_dale_law=force_dale_law
	).build()

	# STEP 2
	model = nt.SequentialRNN(
		layers=[ws_layer],
		device=device,
		foresight_time_steps=x.shape[1] - 1,
		checkpoint_folder=checkpoint_folder,
	)
	model.build()

	# REGULARISATION ON THE CONNECTOME -> Not mandatory
	regularisation = DaleLawL2([ws_layer.forward_weights], alpha=0.5,
							   reference_weights=[nt.init.dale_(torch.zeros(200, 200), inh_ratio=0.5, rho=0.99)])

	# OPTIMIZER FOR GRADIENT DESCENT -> Adam, AdamW, SGD
	optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, maximize=True, weight_decay=0.1)  # Weight decay can be applied to reduce the energy if connectome

	# OPTIMIZER FOR REGULARISATION -> Only needed if regularisation
	optimizer_regul = torch.optim.SGD(regularisation.parameters(), lr=5e-4)

	#STEP 3
	trainer = nt.trainers.RegressionTrainer(
		model,
		optimizer=optimizer,
		regularization_optimizer=optimizer_regul,
		criterion=nt.losses.PVarianceLoss(),
		regularisation=regularisation,
		metrics=[regularisation]
	)
	trainer.train(
		DataLoader(dataset, shuffle=False, num_workers=0),
		n_iterations=n_iterations,
		exec_metrics_on_train=True
	)
	x_pred = torch.concat([
		torch.unsqueeze(x[:, 0].clone(), dim=1).to(model.device),
		model.get_prediction_trace(torch.unsqueeze(x[:, 0].clone(), dim=1))
	], dim=1)
	out = {
		"W": ws_layer.forward_weights.detach().cpu().numpy(),
		"mu": ws_layer.mu.detach().numpy(),
		"r": ws_layer.r.detach().numpy(),
		"tau": ws_layer.tau.detach().numpy(),
		"x_pred": torch.squeeze(x_pred).detach().numpy().T,
		"original_time_series": dataset.original_series,
	}
	return out

It is now time to call the function and train our data!

In [ ]:
initial_forward_weight = nt.init.dale_(torch.zeros(200, 200), inh_ratio=0.5, rho=0.2)

result = train_with_params(
	sigma=25,
	learning_rate=1e-2,
	n_iterations=500,
	forward_weights=initial_forward_weight,
	std_weights=1,
	dt=0.02,
	mu=0.0,
	mean_mu=0,
	std_mu=1,
	r=0.1,
	mean_r=0.2,
	std_r=0,
	tau=0.1,
	learn_mu=True,
	learn_r=True,
	learn_tau=True,
	device=torch.device("cpu"),
	hh_init="inputs",
	force_dale_law=False
	)

---
### 6 - Visualise data

Now, we might want to extract the trained parameters such as the connectome to visualise them. We will slightly modify our function train_with params to extract the trained value. We will use the class Visualise that allows us to plot our data in numerous way such as heatmap or ridgeplot.

In [ ]:
pred_viz = nt.Visualise(
	result["x_pred"].T,
	shape=nt.Size(
		[
			nt.Dimension(None, nt.DimensionProperty.TIME, "Time [s]"),
			nt.Dimension(None, nt.DimensionProperty.NONE, "Activity [-]"),
		]
	)
)

pred_viz.plot_timeseries_comparison_report(
	result["original_time_series"].T,
	title=f"Prediction",
	show=True,
)

In [ ]:
VisualiseKMeans(result["x_pred"], nt.Size([nt.Dimension(200, nt.DimensionProperty.NONE, "Neuron [-]"), nt.Dimension(406, nt.DimensionProperty.TIME, "time [s]")])).heatmap(show=True)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(result["W"].T, cmap="RdBu_r")
plt.colorbar()
plt.show()

---
### 7 - Stacking multiple layers

If the goal of the training is to extract a realistic connectome, then it is important to use one layer as shown previously. However, we can stack multiple layers to reproduce/predict the time step. Hence, if the goal is to reproduce and/or predict data without extracting a connectome, then adding layers might be a good idea since it is possible to get more abstract representation of the data. Therefore, it is possible to better approximate complex functions that might explain the data.

With **NeuroTorch**, it is rather easy to add *N* supplementary layers. To add more layers :

1- Chose how the hidden states are initialized.
2- Create *N* objects <code>WilsonCowanLayer</code> (If a deepcopy is made, you must change the name of the new one!)
3- Apply regularisation on the different connectome (if desired)

#### Step 1 : hidden states

When dealing with multiple layers, you must use hidden states to ensure that your Euler integration is done properly. In section 1, we have shown the equation that was implemented in NeuroTorch. However, the equation that is *actually* implemented is slightly different to ensure that hidden states and multi-layers training is working correctly. The *actual* equation is :

# <center> $\vec{x}_{(t+1)} = \vec{h}_{(t)}\left(1 - \frac{\Delta t}{\tau}\right) + \frac{\Delta t}{\tau}\left(\left(1 - \vec{r}\odot\vec{h}_{(t)}\right)\odot \sigma\left(\vec{h}_{(t)}\mathbf{W}^{\text{rec}}+\vec{x}_{(t)}\mathbf{W} - \vec{\mu}\right)\right)$

The new terms can be defined in the following way :

- $\mathbf{W}^{\text{rec}}$ is the recurrent matrix of size (output x output)
- $\vec{h}_{(t)}$ is the hidden state at a time $t$

With **NeuroTorch**, you can initialise the hidden states with zeros, with random numbers or with the actual inputs. In facts, when working with one layer, the hidden states are initialise with the inputs. Therefore, $\vec{h}_{(t)} = \vec{x}_{(t)}$ and $\mathbf{W}^{\text{rec}}$ is a matrix fill with zeros. When working with multiple layers, the output of layer is the input of the next one and so on. In this tutorial, we will initialise the hidden states with the inputs. We pass it as an argument to each Wilson-Cowan layers.

#### Step 2 : Create *N* layers

You can stack up layers by passing them in the list in the Sequential model. In this tutorial, we use two layers

#### Step 3 : Apply regularisation

You can apply the regularisation by applying the forward weight of the new model to the regularisation

When combining those 3 steps, the code becomes :

In [ ]:
def train_with_params(
		sigma: float = 15.0,
		learning_rate: float = 1e-3,
		n_iterations: int = 100,
		forward_weights: Optional[torch.Tensor or np.ndarray] = None,
		std_weights: float = 1,
		dt: float = 1e-3,
		mu: Optional[float or torch.Tensor or np.ndarray] = 0.0,
		mean_mu: Optional[float] = 0.0,
		std_mu: Optional[float] = 1.0,
		r: Optional[float or torch.Tensor or np.ndarray] = 1.0,
		mean_r: Optional[float] = 1.0,
		std_r: Optional[float] = 1.0,
		tau: float = 1.0,
		learn_mu: bool = False,
		learn_r: bool = False,
		learn_tau: bool = False,
		device: torch.device = torch.device("cpu"),
		hh_init: str = "inputs",
		checkpoint_folder="./checkpoints",
		force_dale_law: bool = True
):
	dataset = WSDataset(sample_size=200, smoothing_sigma=sigma, device=device)
	x = dataset.full_time_series
	ws_layer = WilsonCowanLayer(
		x.shape[-1], x.shape[-1],
		forward_weights=forward_weights,
		std_weights=std_weights,
		forward_sign=0.5,
		dt=dt,
		r=r,
		mean_r=mean_r,
		std_r=std_r,
		mu=mu,
		mean_mu=mean_mu,
		std_mu=std_mu,
		tau=tau,
		learn_r=learn_r,
		learn_mu=learn_mu,
		learn_tau=learn_tau,
		hh_init=hh_init,
		device=device,
		name="WilsonCowan_layer1",
		force_dale_law=force_dale_law
	).build()

	ws_layer_2 = deepcopy(ws_layer)
	ws_layer_2.name = "WilsonCowan_layer2"

	model = nt.SequentialRNN(
		layers=[ws_layer, ws_layer_2],
		device=device,
		foresight_time_steps=x.shape[1] - 1,
		checkpoint_folder=checkpoint_folder,
	)
	model.build()

	regularisation = DaleLawL2([ws_layer.forward_weights, ws_layer_2.forward_weights], alpha=0.5,
							   reference_weights=[nt.init.dale_(torch.zeros(200, 200), inh_ratio=0.5, rho=0.99)])
	optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, maximize=True, weight_decay=0.1)  # Weight decay can be applied to reduce the energy if connectome

	optimizer_regul = torch.optim.SGD(regularisation.parameters(), lr=5e-4)
	trainer = nt.trainers.RegressionTrainer(
		model,
		optimizer=optimizer,
		regularization_optimizer=optimizer_regul,
		criterion=nt.losses.PVarianceLoss(),
		regularisation=regularisation,
		metrics=[regularisation]
	)
	trainer.train(
		DataLoader(dataset, shuffle=False, num_workers=0),
		n_iterations=n_iterations,
		exec_metrics_on_train=True
	)
	x_pred = torch.concat([
		torch.unsqueeze(x[:, 0].clone(), dim=1).to(model.device),
		model.get_prediction_trace(torch.unsqueeze(x[:, 0].clone(), dim=1))
	], dim=1)
	out = {
		"W": ws_layer.forward_weights.detach().cpu().numpy(),
		"mu": ws_layer.mu.detach().numpy(),
		"r": ws_layer.r.detach().numpy(),
		"tau": ws_layer.tau.detach().numpy(),
		"x_pred": torch.squeeze(x_pred).detach().numpy().T,
		"original_time_series": dataset.original_series,
	}
	return out

In [ ]:
initial_forward_weight = nt.init.dale_(torch.zeros(200, 200), inh_ratio=0.5, rho=0.2)

result = train_with_params(
	sigma=25,
	learning_rate=1e-2,
	n_iterations=500,
	forward_weights=initial_forward_weight,
	std_weights=1,
	dt=0.02,
	mu=0.0,
	mean_mu=0,
	std_mu=1,
	r=0.1,
	mean_r=0.2,
	std_r=0,
	tau=0.1,
	learn_mu=True,
	learn_r=True,
	learn_tau=True,
	device=torch.device("cpu"),
	hh_init="inputs",
	force_dale_law=False
	)

In [ ]:
pred_viz = nt.Visualise(
	result["x_pred"].T,
	shape=nt.Size(
		[
			nt.Dimension(None, nt.DimensionProperty.TIME, "Time [s]"),
			nt.Dimension(None, nt.DimensionProperty.NONE, "Activity [-]"),
		]
	)
)

pred_viz.plot_timeseries_comparison_report(
	result["original_time_series"].T,
	title=f"Prediction",
	show=True,
)

---
### 8 - Callbacks

Callbacks are useful to improve the training. For instance, here's two useful callbacks.

##### Save the best training : checkpoint manager

This first callback allows you to save the best training. To maximize the speed of training, you can specify that this callback only applied for the iterations near the end. For instance :

In [ ]:
	checkpoint_manager = nt.CheckpointManager(
		checkpoint_folder,
		metric="train_loss",
		minimise_metric=False,
		save_freq=-1,
		save_best_only=True,
		start_save_at=int(0.98 * n_iterations),
	)

Here, we specify that we want to apply this callback to the train loss (the pVar) that is being maximised. Also, with <code>save_freq=-1</code> we specify that we want to save the last iteration. Also, we start saving after 98% of the iterations are done.

##### Learning rate scheduling

This callback is to reduce the learning rate once we arrive close to a certain loss. The pVar tends to be unstable near 0.95 especially with a big learning rate. Hence, it is usefull to be able to reduce gradually the learning rate once we reach a high performance since the remaining adjustment are minimal in order to achieve a better loss. This callback can be combined with the previous one by adding them in a list. This can be done in the following way:

In [ ]:
	callbacks = [
		LRSchedulerOnMetric(
			'train_loss',
			metric_schedule=np.linspace(0.97, 1.0, 100),
			min_lr=learning_rate/10,
			retain_progress=True,
		),
		checkpoint_manager,
		convergence_time_getter,
		EarlyStoppingThreshold(metric='train_loss', threshold=0.999, minimize_metric=False),
	]

We schedule the train loss to reduce once it is between 0.97 and 1. We reduce the learning rate to a minimum of <code> learning_rate/10 </code>. Also, you can apply an early stopping to automatically stop the training once the loss is at a certain value. Finally, <code> convergence_time_getter </code> allows you to know the optimal time needed to obtain a certain loss.


---

### 9 - Alternative way to enforce Dale's law


Previously, we have shown that we can use a regularisation on the connectome to enforce Dale's law. This method is great if you're looking for a specific ratio of excitatory and inhibatory neurons and if computation time is a concern. However, one of the issue with this method, is that the ratio has to be set manually. This alternative way to enforce Dale's law allows you to let the AI "choose" a ratio that optimise best the loss function. In NeuroTorch, this alternative method is applied in the following way:

<center> $\mathbf{W} = \mathbf{W_{\text{weight}}} \odot \tanh({\vec{w}})$ </center>
    

Where :
- $\mathbf{W}$ is the full connectome
- $\mathbf{W_{\text{weight}}}$ is the weight of the connexion in absolute value. The sign of those connexion is not consider in this matrix
- $\vec{w}$ Is a vector that contain the information about the sign of each neuron. A $\tanh$ is applied so that this vector is define between -1 and 1. A $\tanh$ is also use because, as a continuous function, it is easier to train since vanishing gradient is less of a problem. 

To use this method, one simply needs to add an argument in the <code>kwargs</code> of the layer. For instance, here's an example :

In [ ]:
ws_layer = WilsonCowanLayer(
		x.shape[-1], x.shape[-1],
		forward_weights=forward_weights,
		std_weights=std_weights,
		forward_sign=0.5,
		dt=dt,
		r=r,
		mean_r=mean_r,
		std_r=std_r,
		mu=mu,
		mean_mu=mean_mu,
		std_mu=std_mu,
		tau=tau,
		learn_r=learn_r,
		learn_mu=learn_mu,
		learn_tau=learn_tau,
		hh_init=hh_init,
		device=device,
		name="WilsonCowan_layer1",
		force_dale_law=True
	).build()

<code> forward_sign </code> can be a sign vector or a float (if a specified initial ratio is desired).

**To test all of those features, please use :** <code> main.py </code>! Enjoy!

---